In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io.fits import getdata

import pandas as pd
numaster = getdata('SOC-report-dat.fits')
from astropy.time import Time

cutoff_time = Time('2025-10-01T00:00:00').mjd
start_time = Time('2012-12-31T00:00:00').mjd


In [4]:
numaster.columns

ColDefs(
    name = 'SEQUENCEID'; format = '11A'
    name = 'NAME'; format = '29A'
    name = 'SURVEY'; format = '4A'
    name = 'AFT_START'; format = '17A'
    name = 'AFT_END'; format = '17A'
    name = 'DATE_OBS'; format = '24A'
    name = 'DATE_END'; format = '24A'
    name = 'RA_OBJ'; format = 'D'
    name = 'DEC_OBJ'; format = 'D'
    name = 'RA_PNT'; format = 'D'
    name = 'DEC_PNT'; format = 'D'
    name = 'PA_PNT'; format = 'D'
    name = 'TSTART_01'; format = 'D'
    name = 'TSTOP_01'; format = 'D'
    name = 'TELAPSE_01'; format = 'D'
    name = 'ONTIME_01'; format = 'D'
    name = 'EXPOSURE_A01'; format = 'D'
    name = 'EXPOSURE_B01'; format = 'D'
    name = 'NUPSDOUT'; format = 'D'
    name = 'SOFTVER'; format = '46A'
    name = 'CALDBVER'; format = '8A'
    name = 'DATE_A01'; format = '19A'
    name = 'DATE_AUDIT'; format = '22A'
)

In [19]:
numaster_mjds = []
for t in numaster['DATE_OBS']:
    t0 = Time(t)
    numaster_mjds = np.append(numaster_mjds, t0.mjd)

In [21]:
df = pd.read_csv('stray_light_det_v4a.txt', sep='\s+', names=['SEQID', 'FPM', 'SRC', 'DET0', 'DET1', 'DET2', 'DET3'])

# Filter out cases where the source is really bright:
bright_lim = 100.
df = df[df['SRC']<100.]



df['DET0123'] = df['DET0'] + df['DET1'] + df['DET2'] + df['DET3']
df['DET01'] = df['DET0'] + df['DET1']
df['DET12'] = df['DET1'] + df['DET2']
df['DET23'] = df['DET2'] + df['DET3']
df['DET03'] = df['DET0'] + df['DET3']


names = []
exp = []
ra = []
dec = []
pa = []
mjds = []
for row in df['SEQID']:
    nuind = np.where(numaster['SEQUENCEID'] == f'{row}')

    if (len(nuind[0]) == 0):
        names = np.append(names, '')
        exp = np.append(exp, 0)
        ra = np.append(ra, 0.)
        dec = np.append(dec, 0.)
        pa = np.append(pa, 0.)
        mjds = np.append(mjds, 0.)
    else:
        names = np.append(names, numaster['NAME'][nuind[0]].strip())
        exp = np.append(exp, numaster['EXPOSURE_A01'][nuind[0]])
        ra = np.append(ra,numaster['RA_OBJ'][nuind[0]] )
        dec = np.append(dec,numaster['DEC_OBJ'][nuind[0]] )
        pa = np.append(pa,numaster['PA_PNT'][nuind[0]] )
        mjds = np.append(mjds,numaster_mjds[nuind[0]] )


        
df['NAME'] = names
df['EXPOSURE'] = exp
df['RA'] = ra
df['DEC'] = dec
df['PA'] = pa
df['TIME'] = mjds

# Get rid of stuff with low exposure
df = df[(df['EXPOSURE']>1e3) & (df['TIME']<cutoff_time) & (df['TIME']>start_time)]

# Get rid of stuff near SgrA*, GalSurveys, Gal_Cen, or the GC Magnetar:
#bad_name = ['Sgr', 'GalSurvey', 'Gal_Cen', 'gcmag']
#for bn in bad_name:
#    df = df[~(df['NAME'].str.startswith(bn))]

In [22]:
dfa = df[df['FPM']=='A']
dfb = df[df['FPM']=='B']

running_list = {'A':[], 'B':[]}
# Find the brightest stuff first
for fpm, dfi in zip(['A', 'B'], [dfa, dfb]):
    strong_sl = []
    sl_lim = 7.
    # Find the ones with strong straylight:
    strong_df = dfi[dfi['DET0123'] > sl_lim]
    with open(f'strong_sl_FPM{fpm}.txt', 'w') as f:
        for ind, row in strong_df.iterrows():
            outstring = f"??, {row['SEQID']}, {fpm}, {row['NAME']}, {row['EXPOSURE']}, {row['RA']}, {row['DEC']}, {row['PA']}"
            f.write(outstring+'\n') 
            running_list[fpm] = np.append(running_list[fpm], row['SEQID'])

            
# Trim out bright stuff
dfa = dfa[dfa['DET0123'] < sl_lim].copy().reset_index(drop=True)
dfb = dfb[dfb['DET0123'] < sl_lim].copy().reset_index(drop=True)

In [23]:


# Iteration step: find stuff above 5-sigma

sig_limit = 3.0
for det_comb in ['DET0123', 'DET12', 'DET23', 'DET01', 'DET03', 'DET1', 'DET2', 'DET3', 'DET0']:
    use = {'A':np.full(len(dfa), True, dtype=bool),
           'B':np.full(len(dfb), True, dtype=bool)}

    for fpm, dfi in zip(['A', 'B'], [dfa, dfb]):
        with open(f'sig_cuts_{det_comb}{fpm}.txt', 'w') as f:
            for iter in range(3):
                dfi['SIG'] = (dfi[det_comb] - dfi[det_comb].mean()) / dfi[det_comb].std()
                high_sig = dfi[dfi['SIG'] > sig_limit]
                for ind, row in high_sig.iterrows():
                    use[fpm][ind] = False
                    if row['SEQID'] not in running_list[fpm]:
                        running_list[fpm] = np.append(running_list[fpm], row['SEQID'])
                        outstring = f"??, {row['SEQID']}, {fpm}, {row['NAME']}, {row['EXPOSURE']}, {row['RA']}, {row['DEC']}, {row['PA']}"

                        f.write(outstring+'\n')
                dfi = dfi[dfi['SIG']<sig_limit].copy()

    # Now get rid of everything that you've already used for the next round of searching.
    dfa = dfa[use['A']].copy().reset_index(drop=True)
    dfb = dfb[use['B']].copy().reset_index(drop=True)